## Librerías

In [7]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Note: you may need to restart the kernel to use updated packages.


In [8]:
%run "../funciones.ipynb"

In [9]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l

data = pd.read_parquet(dataset_clase_ternaria_l)

## Valores null por variable

In [12]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    breakpoints = sorted(set(
    [edge.left for edge in bin_edges if isinstance(edge, pd.Interval)] + 
    [edge.right for edge in bin_edges if isinstance(edge, pd.Interval)]))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [13]:
# Ingresar mes train y mes score
train_data = data[data['foto_mes'] <= 202104]
score_data = data[data['foto_mes'] == 202106]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)
pd.set_option('display.max_rows', None)
comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
cmobile_app_trx,18.668335,0.000000,18.668335
tmobile_app,18.668335,0.000000,18.668335
Master_fultimo_cierre_lag_2,19.133309,11.636017,7.497291
Master_msaldodolares_lag_2,19.074396,11.602659,7.471737
Master_msaldototal_lag_2,19.074396,11.602659,7.471737
Master_msaldopesos_lag_2,19.074396,11.602659,7.471737
Master_mpagado_lag_2,19.074396,11.602659,7.471737
Master_mlimitecompra_lag_2,19.074396,11.602659,7.471737
Master_mpagominimo_lag_2,19.074396,11.602659,7.471737
Master_fechaalta_lag_2,19.074396,11.602659,7.471737


## Análisis drifting con PSI

In [14]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)


/tmp/ipykernel_5190/2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/tmp/ipykernel_5190/2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/tmp/ipykernel_5190/2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/tmp/ipykernel_5190/2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/tmp/ipykernel_5190/2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/tmp/ipykernel_5190/2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
/tmp

In [15]:
f = psi_df['psi'] > 0.2
lista_variables_con_drift = psi_df.loc[f, 'feature'].to_list()

In [16]:
lista_variables_con_drift

['Visa_Fvencimiento_delta_2',
 'Visa_Finiciomora_delta_1',
 'Visa_Finiciomora_delta_2',
 'Master_fechaalta_delta_2',
 'Master_fechaalta_delta_1',
 'Visa_Fvencimiento_delta_1',
 'Master_fultimo_cierre_delta_1',
 'Master_Fvencimiento_delta_2',
 'mpayroll',
 'mpayroll2',
 'ccajas_depositos_delta_1',
 'mprestamos_prendarios',
 'Master_Fvencimiento_delta_1',
 'Master_fultimo_cierre_lag_1',
 'mcajeros_propios_descuentos',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Visa_fechaalta_delta_2',
 'Visa_fultimo_cierre_lag_2',
 'Visa_fechaalta_delta_1',
 'mprestamos_prendarios_lag_1',
 'mpayroll2_lag_1',
 'mpayroll_lag_2',
 'mpayroll_lag_1',
 'mprestamos_prendarios_lag_2',
 'mpayroll2_lag_2',
 'mcajeros_propios_descuentos_lag_2',
 'mcajeros_propios_descuentos_lag_1',
 'Visa_fultimo_cierre_delta_1',
 'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Visa_madelantopesos_lag_1',
 'Visa_madelantopesos_lag_2',
 'Visa_Finiciomora_lag_2',
 'cmobile_app_trx'

In [17]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=10, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [np.float64(-1.001), np.float64(9.0), np.float64(15.0), np.float64(18.0), np.float64(19.0), np.float64(20.0), np.float64(21.0), np.float64(24.0), np.float64(46.0), np.float64(52.0), np.float64(207.0)]
Frecuencia Esperada: [1851 1768 1347 1821 1275 2031 2676 2082 2016 1972]
Frecuencia Actual: [ 73   0   0   0   0   0 306  39 128 117]


## Analisis drifting con modelo

In [38]:
# 2- Para el caso de que el dataset sea con feature engineering incluidoT_Visa_normal
data['T_Visa_normal'] = data['T_Visa_normal'].astype(bool)
data['T_Master_normal'] = data['T_Master_normal'].astype(bool)


In [39]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000


data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001


In [40]:
# 4 - binarización de la clase ternaria

data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [41]:
# 5 Especificar mes de train y test

df_train = data[data['foto_mes']<=202104]
df_test = data[data['foto_mes']==202106]


In [42]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [43]:
w_train = df_train.loc[X_train.index, 'clase_peso']


In [44]:
best_params = {'num_leaves': 40,
 'learning_rate': 0.022596031021514126,
 'min_data_in_leaf': 1692,
 'feature_fraction': 0.8060065372089812,
 'bagging_fraction': 0.8663914349386255}

In [45]:
train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)


params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': 0,
'learning_rate': 0,
'min_data_in_leaf': 0,
'feature_fraction': 0,
'bagging_fraction': 0,
'seed': semillas[0],
'verbose': 0
}
    #actualización con valores optuna
params.update(best_params)

if params['num_leaves'] == 0:
    print ('Error falta actualización de parámetros')


model = lgb.train(params,
                train_data,
                num_boost_round=299)



In [46]:
model_df = model.trees_to_dataframe()

In [47]:
model_df[model_df['split_feature'] == 'cpayroll_trx'].sort_values('threshold')

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
21851,276,6,276-S20,276-S25,276-L21,276-S13,cpayroll_trx,54.676498,1.000000e-35,<=,left,None,0.001696,1328.6300,185547
21720,274,4,274-S33,274-S34,274-L34,274-S3,cpayroll_trx,37.698898,1.000000e-35,<=,left,None,-0.007292,789.0540,223333
21325,269,4,269-S31,269-S32,269-L32,269-S7,cpayroll_trx,38.985802,1.000000e-35,<=,left,None,-0.007628,806.5320,223333
21264,269,13,269-S22,269-S28,269-S23,269-S21,cpayroll_trx,56.545399,1.000000e-35,<=,left,None,0.001185,10757.6000,684418
21131,267,8,267-S26,267-S27,267-L27,267-S14,cpayroll_trx,47.449902,1.000000e-35,<=,left,None,0.012668,444.5440,73406
21116,267,7,267-S17,267-L4,267-L18,267-S5,cpayroll_trx,54.245701,1.000000e-35,<=,left,None,0.003588,332.8120,73468
20731,262,7,262-S27,262-L4,262-L28,262-S5,cpayroll_trx,56.673698,1.000000e-35,<=,left,None,0.003109,339.4100,74902
20651,261,15,261-S25,261-S26,261-L26,261-S24,cpayroll_trx,52.370300,1.000000e-35,<=,left,None,0.002425,5193.1400,519420
20580,260,8,260-S37,260-S38,260-L38,260-S36,cpayroll_trx,62.789200,1.000000e-35,<=,left,None,0.020849,250.5030,49239
20390,258,9,258-S15,258-S16,258-S17,258-S14,cpayroll_trx,70.046700,1.000000e-35,<=,left,None,0.001398,15834.3000,1279340


## Analisis SHAPs

Calculamos la importancia de las variables usando SHAP


In [48]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap_df = pd.DataFrame(shap_values, columns = X_train.columns)

shap_importancia = pd.DataFrame(np.abs(shap_values).mean(0), columns=['SHAP Importance'])
shap_importancia['Feature'] = X_train.columns
shap_importancia.sort_values('SHAP Importance', ascending=False, inplace=True)
shap_importancia



/home/n_alba_dscience/.venv/lib/python3.12/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


,SHAP Importance,Feature
1,2.542478,foto_mes
51,0.232301,cpayroll_trx
107,0.201367,ctrx_quarter
52,0.167156,mpayroll
582,0.131259,Visa_fultimo_cierre_delta_2
18,0.109947,mcaja_ahorro
33,0.106215,mprestamos_personales
28,0.102499,mtarjeta_visa_consumo
581,0.100374,Visa_fultimo_cierre_delta_1
4,0.095864,internet


In [49]:
shap_importancia.columns

Index(['SHAP Importance', 'Feature'], dtype='object')

In [50]:
lista_shap_unimportant = shap_importancia.loc[shap_importancia['SHAP Importance']==0,'Feature'].to_list()

In [51]:
lista_para_sacar = []
lista_para_transformar = []

for x in lista_variables_con_drift:
    if x in lista_shap_unimportant:
        lista_para_sacar.append(x)
    if x not in lista_shap_unimportant:
        lista_para_transformar.append(x)

In [52]:
lista_para_transformar

['Visa_Fvencimiento_delta_2',
 'Visa_Finiciomora_delta_1',
 'Master_fechaalta_delta_2',
 'Master_fechaalta_delta_1',
 'Visa_Fvencimiento_delta_1',
 'Master_fultimo_cierre_delta_1',
 'Master_Fvencimiento_delta_2',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_Fvencimiento_delta_1',
 'Master_fultimo_cierre_lag_1',
 'Visa_mlimitecompra',
 'Visa_fultimo_cierre',
 'Visa_madelantopesos',
 'Visa_Finiciomora',
 'Visa_fechaalta_delta_2',
 'Visa_fultimo_cierre_lag_2',
 'Visa_fechaalta_delta_1',
 'mprestamos_prendarios_lag_1',
 'mpayroll_lag_2',
 'mpayroll_lag_1',
 'mprestamos_prendarios_lag_2',
 'Visa_fultimo_cierre_delta_1',
 'Master_Finiciomora',
 'Master_fultimo_cierre',
 'Visa_Finiciomora_lag_2',
 'cmobile_app_trx',
 'Visa_fultimo_cierre_lag_1',
 'Master_fultimo_cierre_lag_2',
 'Master_fultimo_cierre_delta_2',
 'Visa_fultimo_cierre_delta_2',
 'internet',
 'mcomisiones_mantenimiento_lag_1',
 'mpayroll_delta_1',
 'ccomisiones_otras_delta_2',
 'mcomisiones_mantenimiento',
 'mrentabilidad_annu

In [53]:
lista_para_sacar

['Visa_Finiciomora_delta_2',
 'mpayroll2',
 'ccajas_depositos_delta_1',
 'mcajeros_propios_descuentos',
 'mpayroll2_lag_1',
 'mpayroll2_lag_2',
 'mcajeros_propios_descuentos_lag_2',
 'mcajeros_propios_descuentos_lag_1',
 'Visa_madelantopesos_lag_1',
 'Visa_madelantopesos_lag_2',
 'cforex_lag_2',
 'Master_Fvencimiento_lag_2',
 'Master_Fvencimiento',
 'Visa_mconsumosdolares_lag_2',
 'mforex_sell_lag_1',
 'cforex_lag_1',
 'Visa_mconsumosdolares_lag_1',
 'cpayroll_trx_delta_1',
 'mtransferencias_recibidas_delta_1',
 'cforex_sell_lag_1',
 'ctransferencias_recibidas_lag_2',
 'cforex_delta_2',
 'Visa_mlimitecompra_delta_2',
 'mforex_sell_delta_2',
 'mtransferencias_recibidas_delta_2',
 'cforex_sell_delta_2']